<a href="https://colab.research.google.com/github/GabrielYamil/TPs_colab/blob/main/TP3/Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gas Monkey Garage

In [17]:
%%writefile GasMonkeyGarage.cpp
#include <iostream>
#include <thread>
#include <chrono>
#include <vector>
#include <memory>
#include <semaphore>
#include <mutex>

using namespace std;

mutex cout_mutex;

class TallerMecanico {
private:
    const int N;

    counting_semaphore<6> espacioPlaya{6};
    counting_semaphore<6> autosParaInspeccion{0};
    counting_semaphore<3> espacioFosa{3};
    counting_semaphore<6> autosParaReparar{0};
    counting_semaphore<2> espacioServicio{2};
    counting_semaphore<1> espacioLavado{1};

public:
    TallerMecanico(int n) : N(n) {}

    void richard(int id) {
        espacioPlaya.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Richard lo ingresa a la playa." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        autosParaInspeccion.release();
    }

    void aaron(int id) {
        autosParaInspeccion.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Aaron lo inspecciona." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioFosa.acquire();
        autosParaReparar.release();
    }

    void charles(int id) {
        autosParaReparar.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] ayudante lo traslada a zona de fosa y Charles lo repara." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioServicio.acquire();
        espacioFosa.release();
    }

    void ayudante(int id) {
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Ayudante lo traslada a zona de servicio y cambia el aceite." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioServicio.release();

        espacioLavado.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Se lava y se lo transporta a playa." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioLavado.release();

        espacioPlaya.release();
    }

    void ingresarAuto(int id) {
        richard(id);
        aaron(id);
        charles(id);
        ayudante(id);
    }

    void iniciar() {
        vector<thread> threads;
        for (int i = 0; i < N; ++i) {
            threads.emplace_back(&TallerMecanico::ingresarAuto, this, i + 1);
        }
        for (auto& t : threads) {
            t.join();
        }
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "Todos los autos fueron atendidos y retirados." << endl;
        }
    }
};

int main(int argc, char* argv[]) {
    if (argc != 2) {
        cerr << "Uso: " << argv[0] << " <N_AUTOS>\n";
        return 1;
    }

    int N = stoi(argv[1]);
    TallerMecanico taller(N);
    taller.iniciar();

    return 0;
}

Writing GasMonkeyGarage.cpp


In [18]:
!ls -la

total 92
drwxr-xr-x 1 root root  4096 May 19 23:39 .
drwxr-xr-x 1 root root  4096 May 19 22:00 ..
drwxr-xr-x 4 root root  4096 May 14 13:38 .config
-rw-r--r-- 1 root root  2972 May 19 23:39 GasMonkeyGarage.cpp
drwxr-xr-x 1 root root  4096 May 14 13:38 sample_data
-rwxr-xr-x 1 root root 73072 May 19 23:30 taller


In [15]:
!rm GasMonkeyGarage.cpp

In [19]:
!g++ -std=c++20 -pthread GasMonkeyGarage.cpp -o taller

In [20]:
!./taller 20

[Auto 5] Richard lo ingresa a la playa.
[Auto 4] Richard lo ingresa a la playa.
[Auto 3] Richard lo ingresa a la playa.
[Auto 2] Richard lo ingresa a la playa.
[Auto 1] Richard lo ingresa a la playa.
[Auto 6] Richard lo ingresa a la playa.
[Auto 5] Aaron lo inspecciona.
[Auto 4] Aaron lo inspecciona.
[Auto 3] Aaron lo inspecciona.
[Auto 2] Aaron lo inspecciona.
[Auto 1] Aaron lo inspecciona.
[Auto 6] Aaron lo inspecciona.
[Auto 3] ayudante lo traslada a zona de fosa y Charles lo repara.
[Auto 6] ayudante lo traslada a zona de fosa y Charles lo repara.
[Auto 4] ayudante lo traslada a zona de fosa y Charles lo repara.
[Auto 4] Ayudante lo traslada a zona de servicio y cambia el aceite.
[Auto 5] ayudante lo traslada a zona de fosa y Charles lo repara.
[Auto 6] Ayudante lo traslada a zona de servicio y cambia el aceite.
[Auto 2] ayudante lo traslada a zona de fosa y Charles lo repara.
[Auto 6] Se lava y se lo transporta a playa.
[Auto 3] Ayudante lo traslada a zona de servicio y cambia el 